In [20]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Leer el documentos

In [21]:
def cargar_documentos(ruta_archivo):
    """Carga documentos en PDF, TXT o DOCX y los convierte en texto."""
    if ruta_archivo.endswith(".pdf"):
        loader = PyPDFLoader(ruta_archivo)
    elif ruta_archivo.endswith(".txt"):
        loader = TextLoader(ruta_archivo)
    elif ruta_archivo.endswith(".docx"):
        loader = Docx2txtLoader(ruta_archivo)
    else:
        raise ValueError("Formato no soportado. Usa PDF, TXT o DOCX.")
    
    documentos = loader.load()
    
    return documentos

In [2]:
# Leer documento pdf
loader = PyPDFLoader("data/1-01-Curso_PLN.pdf")
pages = loader.load()
pages

[Document(metadata={'producer': 'Acrobat Distiller 19.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2024-10-30T15:27:39-05:00', 'author': 'USUARIO', 'moddate': '2024-10-30T15:27:39-05:00', 'title': 'Microsoft PowerPoint - 1-01-Curso_PLN', 'source': 'data/1-01-Curso_PLN.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='30/10/2024\n3011176 - Procesamiento del Lenguaje \nNatural\nFACULTAD DE MINAS\nSede Medellín\nSINTELWEB\nGrupo de Investigación\nSistemas Inteligentes Web\nJAIME ALBERTO GUZMAN LUNA, Ph.D\nInformación de contacto\n\uf06e Jaime Alberto Guzmán Luna\n\uf06e Doctor en Ingeniería de Sistemas e Informática\n\uf06e M.Sc. en Ingeniería de Sistemas e Informática\n\uf06e Especialista en comunicación educativa\n\uf06e Ingeniero Civil\n\uf06e Áreas de trabajo\n\uf06e Inteligencia Artificial\n\uf06e Sistemas de recuperación de información\n\uf06e Planificación automática de procesos \n\uf06e Aprendizaje de máquinas\n\uf06e PLN con aprendiza

# Split document

In [28]:
def split_text(text):
    """Divide el texto en oraciones."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, # Número de caracteres por chunk o fragmento
                                              chunk_overlap=50, # Número de caracteres de traslape entre chunks
                                              length_function=len, # Función para obtener la longitud de un texto
                                              separators=["\n\n", "\n", " "], # Separadores de palabras
                                              )
    textos_fragmentados = text_splitter.split_documents(text)
    
    return textos_fragmentados

# Crear embeddings

Se hará uso de un modelo de Hugging Face all-MiniLM-L6-v2

In [13]:
# Usar modelo de Hugging Face
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\Usuario\Documents\Universidad\Redes Neuronales\Trabajo 4\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Usuario\Documents\Universidad\Redes Neuronales\Trabajo 4\.venv\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Usuario\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need

# Almacenar  embeddings en ChromaDB

In [29]:
def create_vector_store(texts, embeddings):
    """Crea un vector store a partir de una lista de textos."""
    documentos = cargar_documentos(texts)
    textos_fragmentados = split_text(documentos)
    vectorstore = Chroma.from_documents(textos_fragmentados, embeddings, persist_directory="./chroma_db")
    
    return vectorstore

In [30]:
path = "data/1-01-Curso_PLN.pdf"
vectorstore = create_vector_store(path, embeddings)